![](https://storage.googleapis.com/mle-courses-prod/users/61b6fa1ba83a7e37c8309756/private-files/29847d70-c729-11ef-99fc-e9ac20f73f15-LLMs__1_.png)

In [1]:
!pip install transformers==4.43.1

In [2]:
!wget --no-check-certificate \
    https://gist.githubusercontent.com/khacanh/4c4662fa226db87a4664dfc2f70bc63e/raw/5d8a1d890c73a1e92e6898137db28f3dc0676975/kieu.txt \
    -O ./kieu.txt

--2025-05-08 04:18:54--  https://gist.githubusercontent.com/khacanh/4c4662fa226db87a4664dfc2f70bc63e/raw/5d8a1d890c73a1e92e6898137db28f3dc0676975/kieu.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135832 (133K) [text/plain]
Saving to: ‘./kieu.txt’

./kieu.txt          100%[===================>] 132.65K  --.-KB/s    in 0.02s   

2025-05-08 04:18:54 (5.34 MB/s) - ‘./kieu.txt’ saved [135832/135832]



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
corpus = []
f = open("kieu.txt", "r")
for line in f:
  corpus.append(line)

### Xử lý dữ liệu

In [5]:
corpus[:10]

['Trăm năm trong cõi người ta,\n',
 'Chữ tài chữ mệnh khéo là ghét nhau.\n',
 'Trải qua một cuộc bể dâu,\n',
 'Những điều trông thấy mà đau đớn lòng.\n',
 'Lạ gì bỉ sắc tư phong,\n',
 'Trời xanh quen thói má hồng đánh ghen.\n',
 'Cảo thơm lần giỞ trước đèn,\n',
 'Phong tình có lục còn truyền sử xanh.\n',
 'Rằng năm Gia Tĩnh triều Minh,\n',
 'Bốn phương phẳng lặng, hai kinh vững vàng.\n']

In [6]:
model_name = "meta-llama/Llama-3.2-1B"

In [7]:
!pip install datasets

### TODO 1: Lập trình hàm chia mẫu

![](https://storage.googleapis.com/mle-courses-prod/users/61b6fa1ba83a7e37c8309756/private-files/f1faa3a0-c72f-11ef-a3aa-4b5f419a5062-Screen_Shot_2024_12_31_at_11.30.17.png)

In [8]:
def group_sentences(data, group_size=4, stride=2):
    """
    Nhóm các câu thành các mẫu chồng lên nhau có kích thước xác định với khoảng cách chỉ định.

    Tham số:
        data (list of str): Danh sách các câu.
        group_size (int): Số lượng câu trong mỗi nhóm.
        stride (int): Bước nhảy cho các nhóm.

    Kết quả trả về:
        list of str: Danh sách các câu đã được nhóm, nối bằng ký tự xuống dòng.
    """
    grouped_data = []
    for i in range( 0 , len(data) , stride):
      if i + group_size <= len(data):
        tmp  = "\n".join(data[i:i+ group_size]) + "\n"
        grouped_data.append(tmp)
    return grouped_data


In [9]:
from datasets import Dataset, DatasetDict

# Group sentences into overlapping samples of 4 with a stride of 2
grouped_sentences = group_sentences(corpus, 6, 1)

# Split into train and validation datasets
split_index = int(0.8 * len(grouped_sentences))
dataset = DatasetDict({
    "train": Dataset.from_dict({"text": grouped_sentences[:split_index]}),
    "validation": Dataset.from_dict({"text": grouped_sentences[split_index:]})
})


In [10]:
print(dataset['train'][0]['text'])

Trăm năm trong cõi người ta,

Chữ tài chữ mệnh khéo là ghét nhau.

Trải qua một cuộc bể dâu,

Những điều trông thấy mà đau đớn lòng.

Lạ gì bỉ sắc tư phong,

Trời xanh quen thói má hồng đánh ghen.




Test code

In [11]:
import unittest

def test_valid_input():
    """Test with valid input data."""
    data = ["A", "B", "C", "D", "E", "F"]
    result = group_sentences(data, group_size=3, stride=2)
    expected = ["A\nB\nC\n", "C\nD\nE\n"]
    assert result == expected, f"Expected {expected}, but got {result}"

def test_invalid_data_type():
    """Test with non-list data input."""
    try:
        group_sentences("Invalid input")
    except ValueError as e:
        assert str(e) == "Data must be a list of strings"

def test_invalid_element_type():
    """Test with non-string elements in the list."""
    try:
        group_sentences(["A", 123, "C"])
    except ValueError as e:
        assert str(e) == "All elements in data must be strings"

def test_negative_group_size():
    """Test with negative group_size."""
    try:
        group_sentences(["A", "B", "C"], group_size=-1, stride=2)
    except ValueError as e:
        assert str(e) == "Group size and stride must be positive integers"

def test_insufficient_data():
    """Test when data length is less than group size."""
    result = group_sentences(["A", "B"], group_size=4, stride=2)
    assert result == [], f"Expected an empty list, but got {result}"


test_valid_input()
print("✅ test_valid_input passed")

test_invalid_data_type()
print("✅ test_invalid_data_type passed")

test_invalid_element_type()
print("✅ test_invalid_element_type passed")

test_negative_group_size()
print("✅ test_negative_group_size passed")

test_insufficient_data()
print("✅ test_insufficient_data passed")

print("\n🎉 All tests passed successfully!")


✅ test_valid_input passed
✅ test_invalid_data_type passed
✅ test_invalid_element_type passed
✅ test_negative_group_size passed
✅ test_insufficient_data passed

🎉 All tests passed successfully!


***Kết quả mong đợi***

```
✅ test_valid_input passed
✅ test_invalid_data_type passed
✅ test_invalid_element_type passed
✅ test_negative_group_size passed
✅ test_insufficient_data passed

🎉 All tests passed successfully!
```

Thử một số mẫu

In [12]:
grouped_sentences[0]

'Trăm năm trong cõi người ta,\n\nChữ tài chữ mệnh khéo là ghét nhau.\n\nTrải qua một cuộc bể dâu,\n\nNhững điều trông thấy mà đau đớn lòng.\n\nLạ gì bỉ sắc tư phong,\n\nTrời xanh quen thói má hồng đánh ghen.\n\n'

In [13]:
grouped_sentences[1]

'Chữ tài chữ mệnh khéo là ghét nhau.\n\nTrải qua một cuộc bể dâu,\n\nNhững điều trông thấy mà đau đớn lòng.\n\nLạ gì bỉ sắc tư phong,\n\nTrời xanh quen thói má hồng đánh ghen.\n\nCảo thơm lần giỞ trước đèn,\n\n'

In [14]:
import random

In [15]:
from huggingface_hub import login
from google.colab import userdata
# Authenticate with your Hugging Face token
login(
    userdata.get("hugging_face_token")
)

### TODO 2: Hàm tách tokens

In [16]:
from transformers import AutoTokenizer
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Use EOS as PAD token

# Tokenization
def tokenize_function(examples):
    """
    Hàm này thực hiện việc mã hóa (tokenize) dữ liệu văn bản để chuẩn bị đưa vào mô hình ngôn ngữ.

    Tham số:
        examples (dict): Một từ điển chứa các cặp khóa-giá trị,
                         trong đó khóa 'text' chứa danh sách các chuỗi văn bản cần mã hóa.

    Trả về:
        dict: Một từ điển chứa các thành phần sau:
            - input_ids: Danh sách ID của các token đã được mã hóa từ văn bản.
            - attention_mask: Mặt nạ chú ý, giúp mô hình bỏ qua các giá trị đệm (padding).
            - labels: Sao chép từ `input_ids` để sử dụng làm nhãn trong các bài toán huấn luyện có giám sát.
    Chú ý:
        - Tham số `max_length` nên được điều chỉnh phù hợp với kích thước tối đa của mô hình.
    """

    # Mã hóa văn bản
    # - Cắt bớt văn bản nếu nó vượt quá `max_length`.
    # - Thêm giá trị đệm (padding) vào văn bản ngắn hơn `max_length`.
    # - Giới hạn tối đa độ dài chuỗi mã hóa là 64 token.
    tokens = tokenizer(examples["text"], truncation=True, padding='max_length', max_length=128)

    # Sao chép từ `input_ids` để sử dụng làm nhãn trong các bài toán huấn luyện có giám sát.
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


tokenized_datasets = dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/2599 [00:00<?, ? examples/s]

Map:   0%|          | 0/650 [00:00<?, ? examples/s]

In [17]:
len(tokenized_datasets["train"][0]["input_ids"])

128

Code test

In [18]:
sample_dataset = Dataset.from_dict({
    "text": [
        "Trăm năm trong cõi người ta,\n\nChữ tài chữ mệnh khéo là ghét nhau.\n\nTrải qua một cuộc bể dâu,\n\nNhững điều trông thấy mà đau đớn lòng.\n\n"
    ]
})


# Test 1: Keys Exist
def test_tokenization_keys():
    """Ensure output contains input_ids, attention_mask, and labels."""
    sample = sample_dataset[0]
    tokenized = tokenize_function(sample)
    assert "input_ids" in tokenized, "input_ids missing"
    assert "attention_mask" in tokenized, "attention_mask missing"
    print("✅ test_tokenization_keys passed")

# Test 2: Token Length
def test_token_length():
    """Ensure tokenized outputs have the correct length."""
    sample = sample_dataset[0]
    tokenized = tokenize_function(sample)
    assert len(tokenized["input_ids"]) == 64, "input_ids length mismatch"
    assert len(tokenized["attention_mask"]) == 64, "attention_mask length mismatch"
    print("✅ test_token_length passed")

# Test 3: Labels Match Input IDs
def test_labels_match_input_ids():
    """Ensure labels are a copy of input_ids."""
    sample = sample_dataset[0]
    tokenized = tokenize_function(sample)
    assert tokenized["input_ids"][0] == tokenized["labels"][0], "Labels do not match input_ids"
    print("✅ test_labels_match_input_ids passed")

# Test 4: Specific Token IDs
def test_specific_input_ids():
    """Ensure token IDs match the expected output for a specific input text."""
    sample = {
        'text': 'Trăm năm trong cõi người ta,\n\nChữ tài chữ mệnh khéo là ghét nhau.\n\nTrải qua một cuộc bể dâu,\n\nNhững điều trông thấy mà đau đớn lòng.\n\n'
    }
    expected_input_ids = [128000, 1305, 78454, 100631, 70186, 272, 111209, 100524, 9637, 3638, 1163,
        57073, 102312, 111244, 120899, 24040, 89577, 39015, 37224, 10515, 103321,
        382, 1305, 76150, 74625, 100403, 104053, 293, 34550, 294, 101208, 3638,
        112438, 101309, 490, 24976, 102874, 100920, 113829, 15199, 45006, 77,
        99270, 382]

    tokenized = tokenize_function(sample)

    assert tokenized["input_ids"][:44] == expected_input_ids, "Input IDs do not match expected"
    print("✅ test_specific_input_ids passed")

# Run Tests
test_tokenization_keys()
test_token_length()
test_labels_match_input_ids()
test_specific_input_ids()

print("\n🎉 All tests passed successfully!")

✅ test_tokenization_keys passed


AssertionError: input_ids length mismatch

**Kết quả mong đợi**

🎉 All tests passed successfully!

In [19]:
sample_ids = tokenize_function(dataset["train"][0])

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2599
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 650
    })
})

In [21]:
tokens = [tokenizer.decode([token_id]) for token_id in tokenized_datasets["train"]["input_ids"][0]]

print(tokens)

['<|begin_of_text|>', 'Tr', 'ăm', ' năm', ' trong', ' c', 'õi', ' người', ' ta', ',\n\n', 'Ch', 'ữ', ' tài', ' chữ', ' mệnh', ' kh', 'éo', ' là', ' gh', 'ét', ' nhau', '.\n\n', 'Tr', 'ải', ' qua', ' một', ' cuộc', ' b', 'ể', ' d', 'âu', ',\n\n', 'Những', ' điều', ' tr', 'ông', ' thấy', ' mà', ' đau', ' đ', 'ớ', 'n', ' lòng', '.\n\n', 'L', 'ạ', ' gì', ' b', 'ỉ', ' sắc', ' tư', ' phong', ',\n\n', 'Tr', 'ời', ' xanh', ' quen', ' th', 'ói', ' má', ' hồng', ' đánh', ' g', 'hen', '.\n\n', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>', '<|end_of_text|>',

In [22]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal Language Modeling
)

In [23]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
import torch

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
)


### TODO 3: Setup Training

In [24]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./llama-pretrained",  # Thư mục để lưu các điểm kiểm tra (checkpoint) của mô hình
    eval_strategy="epoch",  # Đánh giá mô hình sau mỗi epoch (1 vòng lặp qua toàn bộ dữ liệu huấn luyện)
    save_strategy="epoch",  # Lưu checkpoint của mô hình sau mỗi epoch
    load_best_model_at_end=True,  # Tải mô hình tốt nhất dựa trên tổn thất (loss) khi đánh giá
    metric_for_best_model="eval_loss",  # Sử dụng tổn thất đánh giá (eval loss) để xác định mô hình tốt nhất
    greater_is_better=False,  # Tổn thất càng nhỏ càng tốt (loss thấp hơn là tốt hơn)
    learning_rate=2e-5,  # Tốc độ học (learning rate) của trình tối ưu hóa
    per_device_train_batch_size=2,  # Kích thước batch cho mỗi thiết bị khi huấn luyện
    per_device_eval_batch_size=2,  # Kích thước batch cho mỗi thiết bị khi đánh giá
    num_train_epochs=3,  # Số lượng epoch huấn luyện (số vòng lặp qua toàn bộ tập dữ liệu huấn luyện)
    weight_decay=0.01,  # Hệ số suy giảm trọng số để tránh quá khớp (overfitting)
    logging_dir="./logs",  # Thư mục lưu trữ log để theo dõi quá trình huấn luyện
    logging_steps=10,  # Ghi log sau mỗi 10 bước huấn luyện
    save_total_limit=2  # Chỉ giữ lại 2 checkpoint gần nhất để tiết kiệm dung lượng lưu trữ
)


Test code

In [ ]:
# Expected values for TrainingArguments
assert training_args.output_dir == "./llama-pretrained", "Expected output_dir to be './llama-pretrained'"
assert training_args.evaluation_strategy == "epoch", "Expected evaluation_strategy to be 'epoch'"
assert training_args.save_strategy == "epoch", "Expected save_strategy to be 'epoch'"
assert training_args.load_best_model_at_end is True, "Expected load_best_model_at_end to be True"
assert training_args.metric_for_best_model == "eval_loss", "Expected metric_for_best_model to be 'eval_loss'"
assert training_args.greater_is_better is False, "Expected greater_is_better to be False"
assert training_args.learning_rate == 2e-5, "Expected learning_rate to be 2e-5"
assert training_args.per_device_train_batch_size == 2, "Expected per_device_train_batch_size to be 2"
assert training_args.per_device_eval_batch_size == 2, "Expected per_device_eval_batch_size to be 2"
assert training_args.num_train_epochs == 15, "Expected num_train_epochs to be 15"
assert training_args.weight_decay == 0.01, "Expected weight_decay to be 0.01"
assert training_args.logging_dir == "./logs", "Expected logging_dir to be './logs'"
assert training_args.logging_steps == 10, "Expected logging_steps to be 10"
assert training_args.save_total_limit == 2, "Expected save_total_limit to be 2"
print("🎉 All tests passed!")

**Kết quả mong đợi**

🎉 🎉 All tests passed!

### Tiến hành đào tạo

In [25]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start Training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: teamaiproptit (teamaiproptit-other) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Epoch,Training Loss,Validation Loss
1,0.877700,4.970168
2,0.263000,5.397875
3,0.188700,5.532045


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=3900, training_loss=1.030894261598587, metrics={'train_runtime': 4773.8595, 'train_samples_per_second': 1.633, 'train_steps_per_second': 0.817, 'total_flos': 5827292340682752.0, 'train_loss': 1.030894261598587, 'epoch': 3.0})

### TODO 4: Kiểm tra mất mát trên tập validation

Yêu cầu giá trị mất mát nhỏ hơn 4.0

In [26]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print("Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value}")


Evaluation Results:
eval_loss: 4.970167636871338
eval_runtime: 87.4322
eval_samples_per_second: 7.434
eval_steps_per_second: 3.717
epoch: 3.0


In [27]:
# Check if eval_loss is less than 4.0
exp = 4.0
if 'eval_loss' in eval_results:
    if eval_results['eval_loss'] < exp:
        print("✅ Success: Evaluation loss is less than 4.0!")
    else:
        print("❌ Warning: Evaluation loss is greater than 4.0. Further tuning is needed.")
else:
    print("⚠️ Warning: 'eval_loss' not found in evaluation results.")


❌ Warning: Evaluation loss is greater than 4.0. Further tuning is needed.


### Inference

In [28]:
import torch
from transformers import AutoModelForCausalLM

# Load the best model
best_model_path = trainer.state.best_model_checkpoint
print(f"Best model path: {best_model_path}")

best_model = AutoModelForCausalLM.from_pretrained(
    best_model_path
).to("cuda" if torch.cuda.is_available() else "cpu")

Best model path: ./llama-pretrained/checkpoint-1300


In [30]:
# Generate text using the best model
input_text = "Nếu em"
inputs = tokenizer(input_text, return_tensors="pt").to(best_model.device)

with torch.no_grad():
    outputs = best_model.generate(
        **inputs,
        max_length=300,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text from the best model:")
print(generated_text)


Generated text from the best model:
Nếu em là người phụ bạc,

Thì em cũng lầm chăng chua oan thế này?

Lầm chăng cho đà trông mặt?

Thôi thì thôi có tiếc gì của may!

Sợ khi ong bướm đãi đằng,

Đến điều sống đục, sao bằng thác trong!

Nàng rằng: Tiền định tiên tri,

Phẩm tiên nào rụng rời được này tấc phun.

Mà ta biết điều này hay không?

Trước là Bạc Hạnh, Bạc Bà,

Bên là 4 Kim, 4 Nguyệt, tứ diện bốn bề.

Trước là Vương Quan, sau là Vương,

Năm vương cũng chẳng nặng cân vàng.

Trước là Thúc Sinh, sau là Sinh,

Nhớ khi nước đã xa, khi đất还 gần!

Tú Bà ai nỡ giục giã,

Lỡ chước vàng chè rắp lòng trao tay.

Lỡ chước oan nghiệt thế này,

Chẳng vò mà rối, chẳng đắp mà sầu!

Chúa xuân để thuyền có điều,

Đem vào là che chực sao chè?

Đem vào là che chỞ cho vừa,

Làm cho thấy nàng cũng đang nghi.

Bình thệ từ quan chỉ bề,

Sinh ra bể thẳm, mới rao tâng công.

Bực mình nàng mới thưa:


In [31]:
trainer.push_to_hub(commit_message = "Training completed!")


training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Dat1710/llama-pretrained/commit/317d5fe7546f9bc26ae3a38ebf9ee42fcd89e04d', commit_message='Training completed!', commit_description='', oid='317d5fe7546f9bc26ae3a38ebf9ee42fcd89e04d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Dat1710/llama-pretrained', endpoint='https://huggingface.co', repo_type='model', repo_id='Dat1710/llama-pretrained'), pr_revision=None, pr_num=None)

In [1]:
from transformers import pipeline
# Change `Dat1710` to your Hub username
model_id = "Dat1710/llama-pretrained"
classifier = pipeline("text2text-generation", model=model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

Device set to use cpu
The model 'LlamaForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'

In [2]:
text = "Trăm năm"
preds = classifier(
    text,
    max_length=100,               # Giới hạn độ dài đầu ra
    num_return_sequences=1,       # Trả về 3 kết quả khác nhau
    do_sample=True,               # Bật sampling để kết quả không lặp
    temperature=0.9,              # Điều chỉnh độ ngẫu nhiên (0.7–1.0 là hợp lý)
)

for i, pred in enumerate(preds):
    print(f"[Kết quả {i+1}]: {pred['generated_text']}")

[Kết quả 1]: Trăm năm để một tấm lòng từ đây.

Lòng từ đã ngỏ cửa đóng,

Nàng từ liễu gỡ đến xuân đường,

Chàng theo mòn mỏi bước ra,

Biết đâu hết lý hẹp hòi mày,

Biết đâu thỏ được lớn mà non?

Biết đâu khôn ngoan mà kẻo xuân?

Thôi thuyền thôi chăng mới rồi,

Gọi là sinh trường kỳ lệ nhân thường
